In [36]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn import svm
from pandas import ExcelWriter
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import numpy as np
import re


def xy_sets(line):
    X=None
    y=None
    indexes = map(int, re.findall('\d+', line))
    aux = False
    for i in indexes:
        if aux is False:
            X = scaled[i, :]
            y = target[i]
            aux = True
            continue
        X = np.vstack((X, scaled[i, :]))
        y = np.hstack((y, target[i]))
        
    return X, y, indexes


multiplier = ['0,01', '0,001']
kers = ['linear', 'poly', 'rbf']
dataset_name = ['57EDG', '54BND', 'combined']

for name in dataset_name:
    writer = ExcelWriter(name+'_svm.xlsx', engine='xlsxwriter')
    df = DataFrame()
    temp = DataFrame()
    for m in multiplier:
        for ker in kers:
            # load dataset
            dataset = read_csv(name+'_'+m+'.csv', header=0, index_col=(0,1))
            values = dataset.values

            # split data set into data(input) and target(output)
            data = values[:, 1:] 
            target = values[:, 0]

            # normalize features
            scaled = normalize(data[:, :3], norm='max', axis=0)
            scaled = np.concatenate((scaled, data[:, :4]), axis = 1)

            #create the SVM
            clf = svm.SVC(gamma='auto', kernel=ker)

            scores = np.zeros(20)
            
            cont = 0
            file = open(name+'_kf.txt', 'r')
            while cont < 20:

                train_line = file.readline()
                test_line = file.readline()

                X_train, y_train, train_indexes = xy_sets(train_line)
                X_test, y_test, test_indexes = xy_sets(test_line)

                clf.fit(X_train, y_train)
                prediction = clf.score(X_test, y_test)
                scores[cont] = prediction
                print('Prediction:', prediction)

                cont += 1

            file.close()

            acc = scores.mean()
            std = scores.std()
            col = np.copy(scores)
            col = np.hstack((col, acc))
            col = np.hstack((col, std))

            df.insert(0, ker, col, True)
            last = df.index[-1]
            df = df.rename(index={last: 'std_dev'})
            last = df.index[-2]
            df = df.rename(index={last: 'accuracy'})
            print("Accuracy: %0.2f (+/- %0.2f)" % (acc, std))
            
    if m == '0,01':
        temp = df.copy()
        writer.close()
        continue
    df.to_excel(writer, sheet_name='0,01')
    temp.to_excel(writer, sheet_name='0,001')
    writer.save()
    writer.close()
    

Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 0.3333333333333333
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Accuracy: 0.90 (+/- 0.19)
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 0.3333333333333333
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 0.5
Prediction: 1.0
Prediction: 1.0
Accuracy: 0.84 (+/- 0.21)
Prediction: 1.0
Prediction: 0.6666666666666666
Prediction: 1.0
Prediction: 1.0
Prediction: 1.0
Prediction: 0.3333333333333333
Pr